Measuring Average Points over Expected (APE) for Kickers.

In [7]:
import nfl_data_py as nfl
import numpy as np
import pandas as pd
from matplotlib import *

In [8]:
season_start = 2016
season_end = 2021
min_fga = (season_end+1-season_start)*20

Import and filter PBP data to get kicking data.

In [ ]:
season_data = nfl.import_pbp_data(list(range(season_start, season_end+1)), downcast=True, cache=False, alt_path=None)
season_data = season_data.drop(season_data[season_data['season_type'] == 'POST'].index) # Regular season only.
fg_data = season_data[['kicker_player_name', 'kick_distance', 'field_goal_result', 'fg_prob']] # Limit to relevant parameters.
fg_data = fg_data.dropna(subset=['field_goal_result'], inplace=False) # Remove non-FG plays.
fg_data = fg_data.drop(fg_data[fg_data['field_goal_result'] == 'blocked'].index) # Blocked kicks should not count.

Calculate APE using nflfastR's 'fg_prob' variable. It accounts for field conditions, weather, and kick distance.

In [ ]:
fg_expected_points = fg_data.groupby('kicker_player_name')['fg_prob'].sum()*3
fg_actual_points = fg_data[fg_data['field_goal_result'] == 'made']['kicker_player_name'].value_counts()*3
fg_apes = (fg_actual_points-fg_expected_points)/fg_data['kicker_player_name'].value_counts()
fg_apes = fg_apes.sort_values(ascending=False)
fg_counts = fg_data['kicker_player_name'].value_counts()
fg_pcts = (fg_actual_points/3)/fg_counts

Same process for XPs. APE can be more understandable than XP% - penalties causing more yardage, weather conditions, etc.

In [ ]:
xp_data = season_data[['kicker_player_name', 'kick_distance', 'extra_point_result', 'extra_point_prob']]
xp_data = xp_data.dropna(subset=['extra_point_result'], inplace=False)

In [ ]:
xp_expected_points = xp_data.groupby('kicker_player_name')['extra_point_prob'].sum()
xp_actual_points = xp_data[xp_data['extra_point_result'] == 'good']['kicker_player_name'].value_counts()
xp_apes = (xp_actual_points-xp_expected_points)/xp_data['kicker_player_name'].value_counts()
xp_apes = xp_apes.sort_values(ascending=False)
xp_counts = xp_data['kicker_player_name'].value_counts().astype(int)
xp_pcts = xp_actual_points/xp_counts

In [ ]:
kicker_efficiency = pd.concat([fg_counts, fg_pcts, fg_apes, xp_counts, xp_pcts, xp_apes], axis=1)
kicker_efficiency.columns = ['FGA', 'FG%', 'FG APE', 'XPA', 'XP%', 'XP APE']
kicker_efficiency = kicker_efficiency[kicker_efficiency['FGA']>=min_fga].sort_values(by=['FG APE'], ascending=False)
kicker_efficiency